Let's just try to get this to work.

I need to read in the coordinates of the slab and the oleic ion file

Identify sites on the slab, create a system with the slab and add the oleic ligands to the sites

then I need to output the...



What I could do is try to do the slabs and ligands separately in mbuild and then convert them to separate BGFs and then reparameterize them based on the original structure files and then combine and then convert to LAMMPS input

### lay out the basics

I need to manipulate the molecular geometries and then apply the parameters.

I know how to do this.

In [ ]:
class Atom:
    def __init__(self, index, atom_type, residue_name, chain_num, x, y, z, charge, element):
        self.index = index
        self.atom_type = atom_type
        self.residue_name = residue_name
        self.chain_num = chain_num
        self.coords = [x, y, z]
        self.charge = charge
        self.element = element
        self.bonds = {}

    def add_bond(self, atom, order):
        self.bonds[atom] = order


class BGF:
    def __init__(self, filename):
        self.filename = filename
        self.atoms = {}

    def read_bgf(self):
        with open(self.filename, 'r') as f:
            lines = f.readlines()
            for i in range(len(lines)):
                line = lines[i].strip()
                if line.startswith('ATOM') or line.startswith('HETATM'):
                    index, atom_type, residue_name, chain_num, x, y, z, charge, element = self.parse_atom_line(line)
                    self.atoms[index] = Atom(index, atom_type, residue_name, chain_num, x, y, z, charge, element)
                elif line.startswith('CONECT'):
                    atom1, atom2 = self.parse_conect_line(line)
                    if lines[i + 1].strip().startswith('ORDER'):
                        order = self.parse_order_line(lines[i + 1])
                    else:
                        order = 1
                    self.atoms[atom1].add_bond(self.atoms[atom2], order)
                    self.atoms[atom2].add_bond(self.atoms[atom1], order)

    @staticmethod
    def parse_atom_line(line):
        data = line.split()
        index = int(data[1])
        atom_type = data[2]
        residue_name = data[3]
        chain_num = int(data[4])
        x, y, z = map(float, data[5:8])
        charge = float(data[9])
        element = data[10]
        return index, atom_type, residue_name, chain_num, x, y, z, charge, element

    @staticmethod
    def parse_conect_line(line):
        data = line.split()
        atom1 = int(data[1])
        atom2 = int(data[2])
        return atom1, atom2

    @staticmethod
    def parse_order_line(line):
        return int(line.split()[1])


In [ ]:
class Atom:
    def __init__(self, index, atom_type, residue_name, chain_num, x, y, z, charge, element):
        self.index = index
        self.atom_type = atom_type
        self.residue_name = residue_name
        self.chain_num = chain_num
        self.coords = [x, y, z]
        self.charge = charge
        self.element = element
        self.bonds = []

    def add_bond(self, atom):
        self.bonds.append(atom)

class BGF:
    def __init__(self, filename):
        self.filename = filename
        self.atoms = {}

    def read_bgf(self):
        with open(self.filename, 'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if line.startswith('ATOM') or line.startswith('HETATM'):
                    index, atom_type, residue_name, chain_num, x, y, z, charge, element = self.parse_atom_line(line)
                    self.atoms[index] = Atom(index, atom_type, residue_name, chain_num, x, y, z, charge, element)
                elif line.startswith('CONECT'):
                    atom1, atom2 = self.parse_conect_line(line)
                    self.atoms[atom1].add_bond(self.atoms[atom2])
                    self.atoms[atom2].add_bond(self.atoms[atom1])

    @staticmethod
    def parse_atom_line(line):
        data = line.split()
        index = int(data[1])
        atom_type = data[2]
        residue_name = data[3]
        chain_num = int(data[4])
        x, y, z = map(float, data[5:8])
        charge = float(data[9])
        element = data[10]
        return index, atom_type, residue_name, chain_num, x, y, z, charge, element

    @staticmethod
    def parse_conect_line(line):
        data = line.split()
        atom1 = int(data[1])
        atom2 = int(data[2])
        return atom1, atom2


In [ ]:
class Atom:
    def __init__(self, id, atype, resname, resid, x, y, z, element, chain):
        self.id = id
        self.atype = atype
        self.resname = resname
        self.resid = resid
        self.x = x
        self.y = y
        self.z = z
        self.element = element
        self.chain = chain
        self.bonds = []

class Bond:
    def __init__(self, atom1, atom2, order):
        self.atom1 = atom1
        self.atom2 = atom2
        self.order = order

def parse_order_line(line, atoms):
    split_line = line.split()
    atom1_id = int(split_line[1])
    atom2_id = int(split_line[2])
    order = int(split_line[4])

    atom1 = atoms[atom1_id - 1] # Adjust for zero-based indexing
    atom2 = atoms[atom2_id - 1] # Adjust for zero-based indexing

    bond = Bond(atom1, atom2, order)

    atom1.bonds.append(bond)
    atom2.bonds.append(bond)
    return bond

def parse_bgf_file(file):
    atoms = []
    bonds = []
    for line in file:
        if line.startswith("ATOM"):
            atoms.append(parse_atom_line(line))
        elif line.startswith("ORDER"):
            bonds.append(parse_order_line(line, atoms))
    return atoms, bonds
